In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
import glob
import ast
import numpy as np

# Функция для безопасного преобразования строки в список
def safe_convert_to_list(data_string):
    try:
        return ast.literal_eval(data_string)
    except ValueError:
        return None

# Загрузка всех файлов CSV из указанной папки
file_paths = glob.glob('../preprocessed_data_defected/*.csv')
data_frames = []

for file_path in file_paths:
    df = pd.read_csv(file_path)
    
    # Преобразование строк в списки чисел для каждого измерения
    feature_columns = ['ax', 'ay', 'az', 'gx', 'gy', 'gz']  # Названия колонок с данными датчиков
    for col in feature_columns:
        df[col] = df[col].apply(safe_convert_to_list)

    # Предобработка признаков: извлекаем статистические характеристики из каждого списка
    for col in feature_columns:
        df[f'{col}_mean'] = df[col].apply(np.mean)
        df[f'{col}_std'] = df[col].apply(np.std)
        df[f'{col}_min'] = df[col].apply(np.min)
        df[f'{col}_max'] = df[col].apply(np.max)

    # Включаем класс покрытия как категориальный признак
    df['class'] = df['class'].astype('category')
    data_frames.append(df)

# Объединяем все датафреймы в один
combined_data = pd.concat(data_frames, ignore_index=True)

# Признаки для модели
feature_stats = [f'{col}_{stat}' for col in feature_columns for stat in ['mean', 'std', 'min', 'max']]
features = combined_data[feature_stats + ['class']]

# Целевая переменная
target = combined_data['defected']

# Нормализация данных (кроме категориальных переменных)
scaler = StandardScaler()
features[feature_stats] = scaler.fit_transform(features[feature_stats])

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Обучение модели
model = LogisticRegression()
model.fit(X_train, y_train)

# Предсказание на тестовых данных
predictions = model.predict(X_test)

# Оценка модели
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)


Accuracy: 0.9230769230769231
Classification Report:
               precision    recall  f1-score   support

       False       0.92      1.00      0.96        11
        True       1.00      0.50      0.67         2

    accuracy                           0.92        13
   macro avg       0.96      0.75      0.81        13
weighted avg       0.93      0.92      0.91        13



C:\Users\User\AppData\Local\Temp\ipykernel_58888\131823999.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features[feature_stats] = scaler.fit_transform(features[feature_stats])
